# Prices Module

This notebook demonstrates the price data access utilities from `specparser.amt.prices`.

The prices module provides functions for:
- Loading prices from Parquet files into memory
- Fast O(1) price lookups via dict cache
- SQL queries on price data via DuckDB
- Building price tables for straddle valuation

## Table of Contents

1. [Setup](#1.-Setup) - Imports and configuration
2. [Loading Prices](#2.-Loading-Prices) - `load_all_prices`, `set_prices_dict`
3. [Price Lookups](#3.-Price-Lookups) - `get_price`
4. [Query-Based Access](#4.-Query-Based-Access) - `prices_last`, `prices_query`
5. [Straddle Prices](#5.-Straddle-Prices) - `get_prices`
6. [Cache Management](#6.-Cache-Management) - `clear_prices_dict`, `clear_prices_connection_cache`
7. [Benchmark](#7.-Benchmark) - Performance benchmarks
8. [Summary](#8.-Summary) - Function reference

In [ ]:
# Setup: Imports
import timeit
from specparser.amt import (
    # Price loading and caching (from prices.py)
    load_all_prices,
    set_prices_dict,
    get_price,
    clear_prices_dict,
    clear_prices_connection_cache,
    # Price queries (from prices.py)
    prices_last,
    prices_query,
    # Straddle prices (from prices.py)
    get_prices,
    # Table utilities (from table.py)
    show_table,
    table_to_rows,
)

# Data paths (relative to this notebook)
AMT_PATH = "../data/amt.yml"
PRICES_PATH = "../data/prices.parquet"
CHAIN_PATH = None  # "../data/futs.csv"

---
## 2. Loading Prices

### `load_all_prices(prices_parquet, start_date=None, end_date=None)`

Load prices from a Parquet file into a dict for fast lookups.

The dict maps `"ticker|field|date"` keys to values.

In [ ]:
# Load all prices into memory
prices_dict = load_all_prices(PRICES_PATH)

print(f"Loaded {len(prices_dict):,} price records")

# Show sample keys
sample_keys = list(prices_dict.keys())[:5]
print("\nSample keys:")
for k in sample_keys:
    print(f"  {k} -> {prices_dict[k]}")

In [ ]:
# Load prices for a specific date range
prices_2024 = load_all_prices(PRICES_PATH, "2024-01-01", "2024-12-31")

print(f"Prices for 2024: {len(prices_2024):,} records")

### `set_prices_dict(prices_dict)`

Set the module-level prices dict directly. Useful for:
- Using pre-loaded prices
- Testing with mock data

In [ ]:
# Set the prices dict for use by other functions
set_prices_dict(prices_dict)
print("Prices dict set for module use")

---
## 3. Price Lookups

### `get_price(prices_dict, ticker, field, date_str)`

Get a single price from the prices dict. Returns the value or `"none"` if not found.

In [ ]:
# Look up specific prices
value = get_price(prices_dict, "CL1 Comdty", "PX_LAST", "2024-06-03")
print(f"CL1 Comdty PX_LAST on 2024-06-03: {value}")

# Vol field
vol = get_price(prices_dict, "CL1 Comdty", "1ST_MTH_IMPVOL_100.0%MNY_DF", "2024-06-03")
print(f"CL1 Comdty Vol on 2024-06-03: {vol}")

In [ ]:
# Missing price returns "none"
missing = get_price(prices_dict, "NONEXISTENT", "PX_LAST", "2024-01-01")
print(f"Missing ticker: {missing}")

# Weekend/holiday returns "none"
weekend = get_price(prices_dict, "CL1 Comdty", "PX_LAST", "2024-06-01")  # Saturday
print(f"Weekend date: {weekend}")

---
## 4. Query-Based Access

For ad-hoc queries without loading everything into memory.

### `prices_last(prices_parquet, pattern)`

Get the most recent price for tickers matching a regex pattern.

In [ ]:
# Get last prices for CL (crude oil) tickers
last_prices = prices_last(PRICES_PATH, "^CL.*Comdty")
print(f"Last prices for CL tickers: {len(last_prices['rows'])} rows")
show_table(last_prices)

### `prices_query(prices_parquet, sql)`

Run arbitrary SQL on the prices Parquet file using DuckDB.

In [ ]:
# Custom SQL query
result = prices_query(PRICES_PATH, """
    SELECT ticker, field, date, value
    FROM prices
    WHERE ticker = 'CL1 Comdty'
    AND field = 'PX_LAST'
    AND date >= '2024-06-01'
    AND date <= '2024-06-10'
    ORDER BY date
""")
print("CL1 Comdty prices for first 10 days of June 2024:")
show_table(result)

In [ ]:
# Aggregation query
result = prices_query(PRICES_PATH, """
    SELECT 
        ticker,
        COUNT(*) as n_prices,
        MIN(date) as first_date,
        MAX(date) as last_date
    FROM prices
    WHERE ticker LIKE 'CL%Comdty'
    AND field = 'PX_LAST'
    GROUP BY ticker
    ORDER BY ticker
    LIMIT 10
""")
print("CL ticker coverage:")
show_table(result)

---
## 5. Straddle Prices

### `get_prices(underlying, year, month, i, path, chain_csv, prices_parquet)`

Get daily prices for a straddle from entry to expiry.

This function:
1. Gets the straddle schedule from the AMT file
2. Determines the date range (entry month to expiry month)
3. Looks up vol and hedge prices for each day
4. Returns a table with one row per day

In [ ]:
# Get prices for CL Comdty, June 2024, first straddle component
prices_table = get_prices(
    "CL Comdty",
    2024,
    6,
    0,  # First straddle component
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH
)

print(f"Prices table: {len(prices_table['rows'])} rows")
print(f"Columns: {prices_table['columns']}")
show_table(prices_table)

In [ ]:
# Get prices for an equity (SPX Index)
equity_prices = get_prices(
    "SPX Index",
    2024,
    6,
    0,
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH
)

print(f"SPX Index prices: {len(equity_prices['rows'])} rows")
show_table(equity_prices)

In [ ]:
# Get prices for a currency (EURUSD)
fx_prices = get_prices(
    "EURUSD Curncy",
    2024,
    6,
    0,
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH
)

print(f"EURUSD prices: {len(fx_prices['rows'])} rows")
show_table(fx_prices)

---
## 6. Cache Management

### `clear_prices_dict()`

Clear the module-level prices dict cache.

In [ ]:
clear_prices_dict()
print("Prices dict cache cleared")

### `clear_prices_connection_cache()`

Clear the DuckDB connection cache. Use when Parquet files have been updated.

In [ ]:
clear_prices_connection_cache()
print("DuckDB connection cache cleared")

---
## 7. Benchmark

Performance comparison of price lookup methods.

In [ ]:
def run_bench(func, n_runs=5):
    """Run benchmark without data argument."""
    times = timeit.repeat(func, number=1, repeat=n_runs)
    return min(times) * 1000

N_RUNS = 3

In [ ]:
# Benchmark: Load all prices
clear_prices_dict()

t_load = run_bench(lambda: load_all_prices(PRICES_PATH), N_RUNS)
prices_dict = load_all_prices(PRICES_PATH)

print(f"load_all_prices: {t_load:.1f}ms for {len(prices_dict):,} records")
print(f"Throughput: {len(prices_dict)/t_load:.0f}k records/ms")

In [ ]:
# Benchmark: Dict lookup vs DuckDB query
set_prices_dict(prices_dict)

# Dict lookup (1000 lookups)
def dict_lookups():
    for i in range(1000):
        get_price(prices_dict, "CL1 Comdty", "PX_LAST", "2024-06-03")

# DuckDB query (single query returning same data)
def duckdb_query():
    return prices_query(PRICES_PATH, """
        SELECT value FROM prices 
        WHERE ticker='CL1 Comdty' AND field='PX_LAST' AND date='2024-06-03'
    """)

t_dict = run_bench(dict_lookups, N_RUNS)
t_query = run_bench(duckdb_query, N_RUNS)

print(f"1000 dict lookups: {t_dict:.2f}ms ({1000/t_dict:.0f}k lookups/ms)")
print(f"1 DuckDB query:    {t_query:.2f}ms")
print(f"\nDict is ~{t_query*1000/t_dict:.0f}x faster for single lookups")

In [ ]:
# Benchmark: get_prices for straddle
set_prices_dict(prices_dict)

# Warmup
_ = get_prices("CL Comdty", 2024, 6, 0, AMT_PATH, CHAIN_PATH, PRICES_PATH)

t_get_prices = run_bench(
    lambda: get_prices("CL Comdty", 2024, 6, 0, AMT_PATH, CHAIN_PATH, PRICES_PATH),
    N_RUNS
)

result = get_prices("CL Comdty", 2024, 6, 0, AMT_PATH, CHAIN_PATH, PRICES_PATH)
n_rows = len(result['rows'])

print(f"get_prices: {t_get_prices:.1f}ms for {n_rows} days")
print(f"Throughput: {n_rows/t_get_prices*1000:.0f} rows/sec")

---
## 8. Summary

### Loading Functions

| Function | Description |
|----------|-------------|
| `load_all_prices(parquet, start, end)` | Load prices into dict for fast lookup |
| `set_prices_dict(dict)` | Set module-level prices dict |
| `get_price(dict, ticker, field, date)` | O(1) lookup from prices dict |

### Query Functions

| Function | Description |
|----------|-------------|
| `prices_last(parquet, pattern)` | Get last price for matching tickers |
| `prices_query(parquet, sql)` | Run arbitrary SQL via DuckDB |

### Straddle Functions

| Function | Description |
|----------|-------------|
| `get_prices(underlying, year, month, i, ...)` | Get daily prices for a straddle |

### Cache Functions

| Function | Description |
|----------|-------------|
| `clear_prices_dict()` | Clear in-memory prices cache |
| `clear_prices_connection_cache()` | Clear DuckDB connection cache |

### Key Points

- **Dict lookups are O(1)** - use `load_all_prices` + `get_price` for batch processing
- **DuckDB queries are flexible** - use `prices_query` for ad-hoc analysis
- **`get_prices` builds straddle tables** - returns one row per day with vol/hedge prices
- **Clear caches when data changes** - especially after Parquet file updates